<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dist_sac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [0]:
from ape_x import *
import os
import multiprocessing as mp
import tensorflow as tf
import numpy as np
import time
import shutil

df = "audpred60.csv"

_ = shutil.copy("/content/drive/My Drive/"+df,"/content")
# sac_one_step.ckpt
saver_path = "sac_rsi.ckpt"

restore = False

if restore == True:
  _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".data-00000-of-00001","/content")
  _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".index","/content")
  _ = shutil.copy("/content/drive/My Drive/checkpoint","/content")
  
# if restore == True:
#   _ = shutil.copy("/content/drive/My Drive/model/" + saver_path + ".data-00000-of-00001","/content")
#   _ = shutil.copy("/content/drive/My Drive/model/"+ saver_path + ".index","/content")
#   _ = shutil.copy("/content/drive/My Drive/model/checkpoint","/content")

output_size = 2
  
def actor_work(queues,epsilon, num):
  sess = tf.InteractiveSession()
  actor = Actor(df, 5, num, epsilon, sess,STEP_SIZE=24,OUTPUT_SIZE=output_size, save=False, saver_path=saver_path, restore=restore)
  actor.run(queues, 10, 1000,200,24,n=4)

def leaner_work(queues):
        sess = tf.InteractiveSession()
        leaner = Leaner(df, 5, sess,output_size,'/cpu:0', save=True, saver_path=saver_path, restore=restore,noise=False, norm=False)
        leaner.leaner(queues)

In [0]:
%cd /content

queue = mp.Queue()
ps = [mp.Process(target=leaner_work, args=(queue,))]
for i in range(3):
  epsilon = 0.0 if i == 0 else np.random.rand()
  true = True
  while true:
    if epsilon >= 0.2 or i == 0:
      true = False
    else:
      epsilon = np.random.rand()
  epsilon = 1.0 if i == 1 else epsilon
  ps.append(mp.Process(target=actor_work, args=(queue,epsilon,i)))
            
for p in ps:
    p.start()
    time.sleep(1)

for p in ps:
    p.join()

action probability =  [0.8695652173913043, 0.13043478260869565, 0.0]
trade accuracy =  0.09090909090909091
epoch: 9990, total rewards: -3129.000000, mean rewards: -3265.043478
action probability =  [0.8695652173913043, 0.13043478260869565, 0.0]
trade accuracy =  0.2608695652173913
epoch: 10000, total rewards: -1480.000000, mean rewards: -1544.347826
action probability =  [0.8260869565217391, 0.17391304347826086, 0.0]
trade accuracy =  0.21739130434782608
epoch: 10010, total rewards: -1423.000000, mean rewards: -1484.869565
action probability =  [0.7391304347826086, 0.2608695652173913, 0.0]
trade accuracy =  0.43478260869565216
epoch: 10020, total rewards: -84.000000, mean rewards: -87.652174
action probability =  [0.5652173913043478, 0.43478260869565216, 0.0]
trade accuracy =  0.43478260869565216
epoch: 10030, total rewards: -134.000000, mean rewards: -139.826087
action probability =  [0.4782608695652174, 0.5217391304347826, 0.0]
trade accuracy =  0.21739130434782608
epoch: 10040, tota

In [0]:
from multiprocessing import Process, Queue

def f(q):
  for i in range(100):
    rand = np.random.RandomState()
    if rand.rand() < 0.2:
      action = rand.choice(3, p=(0.2,0.3,0.5))
      print(i)
      print(action)

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    p.join()

In [0]:
a = np.asanyarray([[1,1,1,1],[0,0,0,0]])

In [0]:
b = [i[0][0] for i in a]

In [0]:
import urllib.request
import urllib.parse


url = 'https://www.googleapis.com/drive/v2/files/trash'
f = urllib.request.urlopen(url)